# 문서 기반 답변 Assistants API 

문서 기반으로 RAG를 통해 답변하는 Assistants API의 기본 사용법을 숙지해보자.

In [1]:
from dotenv import load_dotenv
load_dotenv(dotenv_path="../.env")

True

In [2]:
# 패키지 불러오기
import openai

In [3]:
client = openai.OpenAI()

##### 파일 업로드 및 벡터 저장소 추가하기

In [4]:
# 벡터 저장소 생성하기
vector_store = client.beta.vector_stores.create(name="축구 규칙 파일")

In [6]:
# 파일을 벡터 저장소에 올리기
file_streams = open("laws-of-the-game-2024-25-korean-en.pdf", "rb")

file_batch = client.beta.vector_stores.file_batches.upload_and_poll(
  vector_store_id=vector_store.id, files=[file_streams]
)
print(file_batch.status)
print(file_batch.file_counts)

completed
FileCounts(cancelled=0, completed=1, failed=0, in_progress=0, total=1)


##### Create Assistant

In [7]:
instruction = '''
[목적]
이 GPT는 축구 규칙을 상세히 설명해주는 챗봇입니다.

[규칙]
1. 사용자가 축구 규칙에 대해 질문하면 업로드된 파일에서 해당 내용을 찾아 자세히 답변합니다.
2. 파일안에서 마땅한 답을 찾을 수 없거나 축구 규칙에 관한 질문이 아니면 "축구 규칙에 관한 질문만 부탁해요^^" 라고 답해주세요.
3. 답변의 형태는 아래 예시와 같이 해주세요
예시)
-  질문 : 질문 내용
-  답변 : 답변내용 
4. 모든 질문에 한국어로 답변해주세요.
'''

In [8]:
assistant = client.beta.assistants.create(
  name = "축구 규칙 설명 챗봇",
  instructions=instruction,
  model="gpt-4o",
  tools=[{"type": "file_search"}],
  tool_resources={"file_search": {"vector_store_ids": [vector_store.id]}}
)

In [9]:
print(assistant.id)

asst_YsmYXysLY1fj5brl7DsvMh37


##### 생성한 Assistant 업데이트 하기

In [ ]:
# Run this only if you need to update the configuration of the assistant
assistant = client.beta.assistants.update(
  assistant.id,
  instructions=instruction,
  model="gpt-4-1106-preview",
  #model="gpt-3.5-turbo-1106",
  tools=[{"type": "retrieval"}],
  file_ids=[file.id]
)

##### 기존 Assistants 불러오기

In [10]:
# 나의 Assistant 목록 확인
my_assistants = client.beta.assistants.list(
    order="desc",
    limit="20",
)
print(my_assistants.data)

[Assistant(id='asst_YsmYXysLY1fj5brl7DsvMh37', created_at=1737252664, description=None, instructions='\n[목적]\n이 GPT는 축구 규칙을 상세히 설명해주는 챗봇입니다.\n\n[규칙]\n1. 사용자가 축구 규칙에 대해 질문하면 업로드된 파일에서 해당 내용을 찾아 자세히 답변합니다.\n2. 파일안에서 마땅한 답을 찾을 수 없거나 축구 규칙에 관한 질문이 아니면 "축구 규칙에 관한 질문만 부탁해요^^" 라고 답해주세요.\n3. 답변의 형태는 아래 예시와 같이 해주세요\n예시)\n-  질문 : 질문 내용\n-  답변 : 답변내용 \n4. 모든 질문에 한국어로 답변해주세요.\n', metadata={}, model='gpt-4o', name='축구 규칙 설명 챗봇', object='assistant', tools=[FileSearchTool(type='file_search', file_search=FileSearch(max_num_results=None, ranking_options=FileSearchRankingOptions(score_threshold=0.0, ranker='default_2024_08_21')))], response_format='auto', temperature=1.0, tool_resources=ToolResources(code_interpreter=None, file_search=ToolResourcesFileSearch(vector_store_ids=['vs_v4qylbvxNoDRQEGDzsXXuq8t'])), top_p=1.0), Assistant(id='asst_bkaCQFxKb8qIkbTmP0fC6TZZ', created_at=1735308383, description=None, instructions='You are a helpful assistant. If asked about math or computing problems, write and ru

In [11]:
assistant = client.beta.assistants.retrieve("asst_YsmYXysLY1fj5brl7DsvMh37")
print(assistant)

Assistant(id='asst_YsmYXysLY1fj5brl7DsvMh37', created_at=1737252664, description=None, instructions='\n[목적]\n이 GPT는 축구 규칙을 상세히 설명해주는 챗봇입니다.\n\n[규칙]\n1. 사용자가 축구 규칙에 대해 질문하면 업로드된 파일에서 해당 내용을 찾아 자세히 답변합니다.\n2. 파일안에서 마땅한 답을 찾을 수 없거나 축구 규칙에 관한 질문이 아니면 "축구 규칙에 관한 질문만 부탁해요^^" 라고 답해주세요.\n3. 답변의 형태는 아래 예시와 같이 해주세요\n예시)\n-  질문 : 질문 내용\n-  답변 : 답변내용 \n4. 모든 질문에 한국어로 답변해주세요.\n', metadata={}, model='gpt-4o', name='축구 규칙 설명 챗봇', object='assistant', tools=[FileSearchTool(type='file_search', file_search=FileSearch(max_num_results=None, ranking_options=FileSearchRankingOptions(score_threshold=0.0, ranker='default_2024_08_21')))], response_format='auto', temperature=1.0, tool_resources=ToolResources(code_interpreter=None, file_search=ToolResourcesFileSearch(vector_store_ids=['vs_v4qylbvxNoDRQEGDzsXXuq8t'])), top_p=1.0)


##### Thread 생성하기

Thread는 사용자와 챗봇의 대화 내용을 메시지 형태로 저장 및 관리하는 공간  
하나의 Thread에 저장할 수 있는 메시지 수의 제한이 없으며, Thread를 활용하여 별도의 설정 없이 챗봇이 사용자와의 과거 대화를 반영하여 답변하게 된다.

In [12]:
thread = client.beta.threads.create()
print(thread)

Thread(id='thread_NpwjThLKKTgFRWihKeduuIzR', created_at=1737253796, metadata={}, object='thread', tool_resources=ToolResources(code_interpreter=None, file_search=None))


In [13]:
# Thread에 메시지 넣기
message = client.beta.threads.messages.create(
    thread_id=thread.id,
    role="user",
    content="축구장의 크기는?"
)
print(message)

Message(id='msg_aJCgAkPwMoBdrMxp98xNvkVM', assistant_id=None, attachments=[], completed_at=None, content=[TextContentBlock(text=Text(annotations=[], value='축구장의 크기는?'), type='text')], created_at=1737253817, incomplete_at=None, incomplete_details=None, metadata={}, object='thread.message', role='user', run_id=None, status=None, thread_id='thread_NpwjThLKKTgFRWihKeduuIzR')


In [14]:
# Assistant Run 하기
run = client.beta.threads.runs.create(
  thread_id=thread.id,
  assistant_id=assistant.id)

In [15]:
import time

while run.status not in ["completed", "failed"]:
  run = client.beta.threads.runs.retrieve(
    thread_id= thread.id,
    run_id= run.id
  )
  print(run.status)
  time.sleep(5)

completed


In [16]:
# 결과확인(Run complete 후)
messages = client.beta.threads.messages.list(
  thread_id=thread.id
)

for each in messages:
  print(each.role + ": "+each.content[0].text.value)
  print("=========")

assistant: -  질문 : 축구장의 크기는?
-  답변 : 축구장의 크기는 터치라인의 길이가 최소 90m(100 야드)에서 최대 120m(130 야드)이며, 골라인의 길이는 최소 45m(50 야드)에서 최대 90m(100 야드)입니다. 국제경기의 경우, 터치라인은 최소 100m(110 야드)에서 최대 110m(120 야드), 골라인은 최소 64m(70 야드)에서 최대 75m(80 야드)로 설정됩니다【4:0†source】.
user: 축구장의 크기는?


In [17]:
# 질문 추가하기
message = client.beta.threads.messages.create(
    thread_id=thread.id,
    role="user",
    content="오프사이드 룰에 대해 설명해줘"
)

# 메세지 리스트 가져오기
messages = client.beta.threads.messages.list(
  thread_id=thread.id
)

#메세지 프린트
for each in messages:
  print(each.role + ": "+each.content[0].text.value)
  print("=========")

user: 오프사이드 룰에 대해 설명해줘
assistant: -  질문 : 축구장의 크기는?
-  답변 : 축구장의 크기는 터치라인의 길이가 최소 90m(100 야드)에서 최대 120m(130 야드)이며, 골라인의 길이는 최소 45m(50 야드)에서 최대 90m(100 야드)입니다. 국제경기의 경우, 터치라인은 최소 100m(110 야드)에서 최대 110m(120 야드), 골라인은 최소 64m(70 야드)에서 최대 75m(80 야드)로 설정됩니다【4:0†source】.
user: 축구장의 크기는?


In [18]:
run = client.beta.threads.runs.create(
  thread_id=thread.id,
  assistant_id=assistant.id,)

while run.status not in ["completed", "failed"]:

  run = client.beta.threads.runs.retrieve(
    thread_id= thread.id,
    run_id= run.id)

  print(run.status)

  time.sleep(5)

queued
in_progress
completed


In [19]:
# 메세지 리스트 가져오기
messages = client.beta.threads.messages.list(
  thread_id=thread.id
)

#메세지 프린트
for each in messages:
  print(each.role + ": "+each.content[0].text.value)
  print("=========")

assistant: -  질문 : 오프사이드 룰에 대해 설명해줘
-  답변 : 오프사이드 위치에 있는 것만으로는 반칙이 아닙니다. 오프사이드 위치에 있는 선수는 머리, 몸 또는 발의 어느 부분이 상대방 진영에 있고, 상대팀의 마지막에서 두 번째 선수보다 골라인에 더 가까이 있을 때를 말합니다. 여기서 손과 팔은 고려되지 않습니다【8:9†source】.

오프사이드 반칙은 다음의 경우에 발생합니다:
- 오프사이드 위치에 있는 선수가 팀 동료가 패스한 볼을 플레이하거나 터치하여 플레이에 간섭할 때
- 명백하게 상대방의 시선을 차단하거나, 볼을 목적으로 상대방에게 도전하는 경우【8:8†source】.

하지만 직간접 프리킥인 골킥, 스로인, 코너킥 상황에서는 오프사이드 반칙이 적용되지 않습니다【8:12†source】.
user: 오프사이드 룰에 대해 설명해줘
assistant: -  질문 : 축구장의 크기는?
-  답변 : 축구장의 크기는 터치라인의 길이가 최소 90m(100 야드)에서 최대 120m(130 야드)이며, 골라인의 길이는 최소 45m(50 야드)에서 최대 90m(100 야드)입니다. 국제경기의 경우, 터치라인은 최소 100m(110 야드)에서 최대 110m(120 야드), 골라인은 최소 64m(70 야드)에서 최대 75m(80 야드)로 설정됩니다【4:0†source】.
user: 축구장의 크기는?


##### Assistants 삭제

In [20]:
response = client.beta.assistants.delete(assistant.id)
print(response)

AssistantDeleted(id='asst_YsmYXysLY1fj5brl7DsvMh37', deleted=True, object='assistant.deleted')


##### 파일 삭제

In [21]:
# 업로드한 파일 목록 가져오기
file_list = client.files.list()
print(file_list)

SyncCursorPage[FileObject](data=[FileObject(id='file-YXPuQoTc5VP9MuSJhkYU6u', bytes=32890324, created_at=1737252388, filename='laws-of-the-game-2024-25-korean-en.pdf', object='file', purpose='assistants', status='processed', status_details=None), FileObject(id='file-RqDNeXPqNsUP8PV1nmLFpL', bytes=5946083, created_at=1736397329, filename='2020_경제금융용어 700선_게시.pdf', object='file', purpose='assistants', status='processed', status_details=None), FileObject(id='file-BbWhuViwibhJz5VxXDWT7f', bytes=2800, created_at=1735480678, filename='step_metrics.csv', object='file', purpose='fine-tune-results', status='processed', status_details=None), FileObject(id='file-8cfhES3s4PHGUmbtTpwjaN', bytes=2304, created_at=1735478157, filename='my_training_data.jsonl', object='file', purpose='fine-tune', status='processed', status_details=None), FileObject(id='file-JkuxKcmxbqpiLxzi9AtS9U', bytes=1694, created_at=1735477670, filename='my_training_data.jsonl', object='file', purpose='fine-tune', st

In [22]:
# 특정 파일 ID 가져오기
file_id = file_list.data[0].id
print(file_id)

file-YXPuQoTc5VP9MuSJhkYU6u


In [23]:
# 업로드 파일 삭제
response = client.files.delete(file_id)
print(response)

FileDeleted(id='file-YXPuQoTc5VP9MuSJhkYU6u', deleted=True, object='file')
